In [ ]:
import numpy as np
import pandas as pd

train=pd.read_csv("/Users/saksham/Desktop/train.csv")
x_test=pd.read_csv("/Users/saksham/Desktop/test.csv")
Y_train=np.array(train["label"])
X_train=np.array(train.drop(["label"],axis=1))


In [ ]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
a=unpickle("/Users/saksham/Desktop/cifar-10-batches-py/test_batch")
y=np.array(a)
y_test=np.array(a[b'labels'])

In [ ]:

X_train = X_train / 255
_,m_train = X_train.shape

In [ ]:
W1 = np.random.rand(2000, 3072)-0.5 
b1 = np.random.rand(2000, 1)-0.5
W2 = np.random.rand(10, 2000)-0.5
b2 = np.random.rand(10, 1)-0.5 
    

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference
    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2=softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def ideal(Y):
    ideal = np.zeros((Y.size, 10))
    ideal[np.arange(Y.size), Y] = 1
    ideal = ideal.T
    return ideal

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    idealcase = ideal(Y)
    dZ2 = A2 - idealcase
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)

def get_accuracy(predictions, Y):
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, alpha,W1,b1,W2,b2):
    Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
    dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
    W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
    predictions = get_predictions(A2)
    print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

In [ ]:
m=40
for i in range(500):
    for j in range(int(50000/m)):
        W1, b1, W2, b2 = gradient_descent(X_train[m*j:m*(j+1)].T, Y_train[m*j:m*(j+1)].T, 0.0010,W1,b1,W2,b2)

In [ ]:
Z1tr, A1tr, Z2tr, A2tr = forward_prop(W1, b1, W2, b2, X_train.T)
pred=get_predictions(A2tr)
print(get_accuracy(pred,Y_train))

In [ ]:
Z1te, A1te, Z2te, A2te = forward_prop(W1, b1, W2, b2, x_test.T)
pred=get_predictions(Z2te)
print(get_accuracy(pred,y_test))